In [4]:
# !pip install umap-learn

     |████████████████████████████████| 88 kB 3.6 MB/s 
     |████████████████████████████████| 1.1 MB 33.7 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82829 sha256=0eeff38bc03ede8ed8da31bdefa00458f108bb42ba96f21fbcc33f3012d0b1f4
  Stored in directory: /root/.cache/pip/wheels/b3/52/a5/1fd9e3e76a7ab34f134c07469cd6f16e27ef3a37aeff1fe821
  Created wheel for pynndescent: filename=pynndescent-0.5.6-py3-none-any.whl size=53943 sha256=64a8cb2d003ffdaa4a907af7d62a6bd74179d3f00b39a6da182348776ccf1e43
  Stored in directory: /root/.cache/pip/wheels/03/f1/56/f80d72741e400345b5a5b50ec3d929aca581bf45e0225d5c50
Successfully built umap-learn pynndescent


In [6]:
# !pip install spams

     |████████████████████████████████| 2.0 MB 5.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for spams: filename=spams-2.6.5.4-cp37-cp37m-linux_x86_64.whl size=3236646 sha256=4844fc6b541369dac1088314dd6bc19713099245d7a5603e12c4971239318445
  Stored in directory: /root/.cache/pip/wheels/5e/92/17/e718e0e26bfcdd7e81afcf874ac2bc599dd7dd7f3ad78f9d76
Successfully built spams


In [7]:
# !pip install kymatio

     |████████████████████████████████| 93 kB 1.0 MB/s 


In [ ]:
# !pip uninstall sklearn

Found existing installation: sklearn 0.0
Uninstalling sklearn-0.0:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/sklearn-0.0.dist-info/*
Proceed (y/n)? y
  Successfully uninstalled sklearn-0.0


In [ ]:
# !pip install sklearn

  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=ee010d9162273c19c957950d0d89a02b480fdf04f46c12575246d8ad354cdc21
  Stored in directory: /root/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built sklearn


In [1]:
# !pip uninstall scikit-learn

Found existing installation: scikit-learn 1.0.2
Uninstalling scikit-learn-1.0.2:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/scikit_learn-1.0.2.dist-info/*
    /usr/local/lib/python3.7/dist-packages/scikit_learn.libs/libgomp-a34b3233.so.1.0.0
    /usr/local/lib/python3.7/dist-packages/sklearn/*
Proceed (y/n)? y
  Successfully uninstalled scikit-learn-1.0.2


In [2]:
# !pip install scikit-learn

     |████████████████████████████████| 24.8 MB 1.5 MB/s 


In [9]:
# clustering algorithm for label correction 
import numpy as np 
import pandas as pd 
import umap.umap_ as umap 

import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim 

import warnings 
import math 
import tqdm 
import spams 
import time 

from scipy import sparse 
from scipy.linalg import orth 
from scipy.optimize import linear_sum_assignment
# from sklearn.metrics.cluster import supervised 
from sklearn import cluster 
from sklearn.base import BaseEstimator, ClusterMixin 
from sklearn.decomposition import sparse_encode 
from sklearn.decomposition import sparse_encode 
from sklearn.linear_model import orthogonal_mp 
from sklearn.neighbors import kneighbors_graph 
from sklearn.preprocessing import normalize 
from sklearn.utils import check_random_state, check_array, check_symmetric
from sklearn.preprocessing import normalize

from kymatio import Scattering2D 
from sklearn import cluster 
from sklearn.metrics import normalized_mutual_info_score, adjusted_rand_score 
from sklearn.preprocessing import normalize 

# Scraping reddit data 

In [3]:
# randomized delay between requests as a consideration to Reddit's servers and security staff 

import requests 
import time 
import pandas as pd 
from random import randint 

In [4]:
url = "https://www.reddit.com/r/stress.json"

In [5]:
headers = {"User-agent" : ""}
res = requests.get(url, headers=headers)
res.status_code 

200

In [10]:
stress_json = res.json()
stress_json

{'data': {'after': 't3_tzs0l4',
  'before': None,
  'children': [{'data': {'all_awardings': [{'award_sub_type': 'GLOBAL',
       'award_type': 'global',
       'awardings_required_to_grant_benefits': None,
       'coin_price': 150,
       'coin_reward': 0,
       'count': 1,
       'days_of_drip_extension': None,
       'days_of_premium': None,
       'description': 'Thank you stranger. Shows the award.',
       'end_date': None,
       'giver_coin_reward': None,
       'icon_format': None,
       'icon_height': 2048,
       'icon_url': 'https://i.redd.it/award_images/t5_22cerq/klvxk1wggfd41_Helpful.png',
       'icon_width': 2048,
       'id': 'award_f44611f1-b89e-46dc-97fe-892280b13b82',
       'is_enabled': True,
       'is_new': False,
       'name': 'Helpful',
       'penny_donate': None,
       'penny_price': None,
       'resized_icons': [{'height': 16,
         'url': 'https://preview.redd.it/award_images/t5_22cerq/klvxk1wggfd41_Helpful.png?width=16&amp;height=16&amp;auto=webp&

In [11]:
# keys of reddit data 
sorted(stress_json["data"].keys())

['after', 'before', 'children', 'dist', 'geo_filter', 'modhash']

In [12]:
# the after string is the query url that indicateswe want to see the next 25 posts after(predetermined number) 
stress_json["data"]["after"]

't3_tzs0l4'

In [16]:
[post["data"]["name"] for post in stress_json["data"]["children"]]

['t3_fwes89',
 't3_u6wml7',
 't3_u6kp9a',
 't3_u61dcm',
 't3_u5pdeg',
 't3_u5lh2q',
 't3_u5e2ry',
 't3_u4liwf',
 't3_u3zvjb',
 't3_u3b4w8',
 't3_u3ke4p',
 't3_u3iikk',
 't3_u3fx4t',
 't3_u344p5',
 't3_u2uhxi',
 't3_u2mzd2',
 't3_u20vdl',
 't3_u1lss6',
 't3_u19f0m',
 't3_u13s1b',
 't3_u0ksan',
 't3_u0zoub',
 't3_u0pcbs',
 't3_u0e83h',
 't3_u01tzi',
 't3_tzs0l4']

In [18]:
# number of posts in one page 
len(stress_json["data"]["children"])

26

In [19]:
pd.DataFrame(stress_json["data"]["children"])

,kind,data
0,t3,"{'approved_at_utc': None, 'subreddit': 'Stress..."
1,t3,"{'approved_at_utc': None, 'subreddit': 'Stress..."
2,t3,"{'approved_at_utc': None, 'subreddit': 'Stress..."
3,t3,"{'approved_at_utc': None, 'subreddit': 'Stress..."
4,t3,"{'approved_at_utc': None, 'subreddit': 'Stress..."
5,t3,"{'approved_at_utc': None, 'subreddit': 'Stress..."
6,t3,"{'approved_at_utc': None, 'subreddit': 'Stress..."
7,t3,"{'approved_at_utc': None, 'subreddit': 'Stress..."
8,t3,"{'approved_at_utc': None, 'subreddit': 'Stress..."
9,t3,"{'approved_at_utc': None, 'subreddit': 'Stress..."


In [21]:
stress_json["data"]["children"][0]["data"]["selftext"]

"The book is available [Here](https://thewellnesssociety.org/free-coronavirus-anxiety-workbook/) from The Wellness Society. Everyone right now needs a little extra help and hopefully, this e-book can assist some of you in uncovering the toolset you need during this abnormal time, or at least it might help with bridging the gap between now and when you may be able to seek more professional assistance. Obviously, it's not a solution to all problems, and some of you are going to be going through a lot more than others, but I hope many of you can find it useful.\nStay safe, stay healthy."

In [26]:
# functions to automate the data collection process 
def reddit_scrape(url_string, number_of_scrapes, output_list):
  after = None 
  for _  in range(number_of_scrapes):
    if _ == 0:
      print(f"SCRAPING {url_string}\n--------------------------------------------------")
      print("<<<SCRAPING COMMENCED>>>")
      print(f"Downloading Batch {1} of {number_of_scrapes}")
    elif (_+1) % 5 == 0:
      print(f"Downloading batch {(_ + 1)} of {number_of_scrapes}")

    if after == None:
      params = {} 
    else:
      # tells the scraper to get the next set after reddit's after code 
      params = {"after": after}
    res = requests.get(url_string, params=params, headers=headers)
    if res.status_code == 200:
      the_json = res.json() 
      output_list.extend(the_json["data"]["children"])
      after = the_json["data"]["after"]
    else:
      print(res.status_code)
      break 
    time.sleep(randint(1,6))

  print("<<<SCRAPING COMPLETED>>>")
  print(f"Number of posts downloaded: {len(output_list)}")
  print("Number of unique posts: {}".format(len(set([p["data"]["name"] for p in output_list]))))

In [27]:
# call the function 
stress_scraped = [] 
# 50 batches of 26
reddit_scrape("https://www.reddit.com/r/stress.json", 50, stress_scraped)

SCRAPING https://www.reddit.com/r/stress.json
--------------------------------------------------
<<<SCRAPING COMMENCED>>>
<<<SCRAPING COMPLETED>>>
Number of posts downloaded: 1246
Number of unique posts: 820


In [31]:
# function that scrapes the data 
def create_unique_list(original_scrape_list, new_list_name):
  data_name_list =[] 
  for i in range(len(original_scrape_list)):
    # name is the unique name assigned to each post 
    # if the name of the post is not already in data_name_list 
    # then add it to the new_list_name 
    if original_scrape_list[i]["data"]["name"] not in data_name_list:
      new_list_name.append(original_scrape_list[i]["data"])
      data_name_list.append(original_scrape_list[i]["data"]["name"])
  print(f"List now contains {len(new_list_name)} unique scraped posts")

In [32]:
stress_scraped_unique = [] 
create_unique_list(stress_scraped, stress_scraped_unique)

List now contains 820 unique scraped posts


In [37]:
# putting the data into a dataframe saving to csv 
stress = pd.DataFrame(stress_scraped_unique)
# naively assign stress label to every post 
stress["is_stress"] = 1 
stress.head()



,approved_at_utc,subreddit,selftext,author_fullname,saved,mod_reason_title,gilded,clicked,title,link_flair_richtext,...,num_crossposts,media,is_video,call_to_action,poll_data,author_cakeday,crosspost_parent_list,url_overridden_by_dest,crosspost_parent,is_stress
0,None,Stress,The book is available [Here](https://thewellne...,t2_3adaf,False,None,0,False,"Free Covid-19 Anxiety e-Workbook. Please, take...",[],...,1,None,False,None,None,None,None,None,None,1
1,None,Stress,"I feel very stressed, but since I live at my p...",t2_5mvda6wc,False,None,0,False,Is there a way to measure how stressed I am ri...,[],...,0,None,False,None,None,None,None,None,None,1
2,None,Stress,i don’t know how to fix this. every time I fin...,t2_c8sfjcku,False,None,0,False,does anyone else get a headache when they try ...,[],...,0,None,False,None,None,None,None,None,None,1
3,None,Stress,,t2_akwgrq1k,False,None,0,False,r/destress,[],...,0,None,False,None,None,None,None,None,None,1
4,None,Stress,"I constantly get headaches, possibly a neurolo...",t2_7r01zv5o,False,None,0,False,how fo you prevent stress headaches before the...,[],...,0,None,False,None,None,None,None,None,None,1
